In [42]:
import numpy as np

# Create environment

In [43]:
number_of_states = 16
number_of_terminal_states = 2
number_of_non_terminal_states = number_of_states - number_of_terminal_states

In [44]:
max_number_of_actions = 4

In [45]:
number_of_actions_per_non_terminal_state = np.repeat(a = max_number_of_actions, repeats = number_of_non_terminal_states)

In [46]:
number_of_state_action_successor_states = np.repeat(a = 1, repeats = number_of_states * max_number_of_actions)

In [47]:
number_of_state_action_successor_states = np.reshape(a = number_of_state_action_successor_states, newshape = (number_of_states, max_number_of_actions))

In [48]:
state_action_successor_state_indices = np.array([1, 0, 14, 4, 2, 1, 0, 5, 2, 2, 1, 6, 4, 14, 3, 7, 5, 0, 3, 8, 6, 1, 4, 9, 6, 2, 5, 10, 8, 3, 7, 11, 9, 4, 7, 12, 10, 5, 8, 13, 10, 6, 9, 15, 12, 7, 11, 11, 13, 8, 11, 12, 15, 9, 12, 13], dtype = np.int64)

In [49]:
state_action_successor_state_transition_probabilities = np.repeat(a = 1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [50]:
state_action_successor_state_rewards = np.repeat(a = -1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [51]:
state_action_successor_state_indices = np.reshape(a = state_action_successor_state_indices, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_transition_probabilities = np.reshape(a = state_action_successor_state_transition_probabilities, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_rewards = np.reshape(a = state_action_successor_state_rewards, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))

# Set hyperparameters

In [52]:
# Set the number of episodes
number_of_episodes = 10000
# Set the maximum episode length
maximum_episode_length = 200
# Set the number of state-action-value functions
number_of_state_action_value_functions = 3
# Set learning rate alpha
alpha = 0.1
# Set epsilon for our epsilon level of exploration
epsilon = 0.1
# Set discounting factor gamma
discounting_factor_gamma = 1.0

# Create value function and policy arrays

In [53]:
state_action_value_function = np.repeat(a = 0.0, repeats = number_of_state_action_value_functions * number_of_states * max_number_of_actions)
state_action_value_function = np.reshape(a = state_action_value_function, newshape = (number_of_state_action_value_functions, number_of_states, max_number_of_actions))

In [54]:
policy = np.repeat(a = 1.0 / max_number_of_actions, repeats = number_of_non_terminal_states * max_number_of_actions)
policy = np.reshape(a = policy, newshape = (number_of_non_terminal_states, max_number_of_actions))

# Create algorithm

In [55]:
# Set random seed so that everything is reproducible
np.random.seed(seed = 0)

In [56]:
# This function initializes episodes
def initialize_epsiode(number_of_non_terminal_states, max_number_of_actions, state_action_value_function, epsilon, policy):
    # Initial state
    initial_state_index = np.random.randint(low = 0, high = number_of_non_terminal_states, dtype = np.int64) # randomly choose an initial state from all non-terminal states

    # Choose policy for chosen state by epsilon-greedy choosing from the state-action-value function
    policy = epsilon_greedy_policy_from_state_action_function(max_number_of_actions, state_action_value_function, epsilon, initial_state_index, policy)

    # Get initial action
    initial_action_index = np.random.choice(a = max_number_of_actions, p = policy[initial_state_index, :])

    return initial_state_index, initial_action_index, policy

In [57]:
# This function selects a policy greedily from the state-action-value function
def epsilon_greedy_policy_from_state_action_function(max_number_of_actions, state_action_value_function, epsilon, state_index, policy):
    # Combine state-action value functions
    state_action_value_function = np.sum(a = state_action_value_function[:, state_index, :], axis = 0)
    
    # Save max state-action value and find the number of actions that have the same max state-action value
    max_action_value = np.max(a = state_action_value_function)
    max_action_count = np.count_nonzero(a = state_action_value_function == max_action_value)

    # Apportion policy probability across ties equally for state-action pairs that have the same value and zero otherwise
    if max_action_count == max_number_of_actions:
        max_policy_apportioned_probability_per_action = 1.0 / max_action_count
        remaining_apportioned_probability_per_action = 0.0
    else:
        max_policy_apportioned_probability_per_action = (1.0 - epsilon) / max_action_count
        remaining_apportioned_probability_per_action = epsilon / (max_number_of_actions - max_action_count)

    policy[state_index, :] = np.where(state_action_value_function == max_action_value, max_policy_apportioned_probability_per_action, remaining_apportioned_probability_per_action)

    return policy

In [58]:
# This function loops through episodes and updates the policy
def loop_through_episode(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, number_of_state_action_value_functions, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length, state_index, action_index):
    # Loop through episode steps until termination
    for t in range(0, maximum_episode_length):
        # Get reward
        successor_state_transition_index = np.random.choice(a = number_of_state_action_successor_states[state_index, action_index], p = state_action_successor_state_transition_probabilities[state_index, action_index, :])

        reward = state_action_successor_state_rewards[state_index, action_index, successor_state_transition_index]

        # Get next state
        next_state_index = state_action_successor_state_indices[state_index, action_index, successor_state_transition_index]
        
        # Update state action value equally randomly selecting from the state-action-value functions
        updating_state_action_value_function_index = np.random.randint(low = 0, high = number_of_state_action_value_functions, dtype = np.int64)

        state_action_value_function, policy, state_index, action_index = update_state_action_value_function(number_of_non_terminal_states, max_number_of_actions, policy, alpha, epsilon, discounting_factor_gamma, state_index, action_index, reward, next_state_index, updating_state_action_value_function_index, number_of_state_action_value_functions, state_action_value_function);

        if next_state_index >= number_of_non_terminal_states:
            break; # episode terminated since we ended up in a terminal state
        
    return state_action_value_function, policy

In [59]:
# This function updates the state-action-value function
def update_state_action_value_function(number_of_non_terminal_states, max_number_of_actions, policy, alpha, epsilon, discounting_factor_gamma, state_index, action_index, reward, next_state_index, updating_state_action_value_function_index, number_of_state_action_value_functions, state_action_value_function):
    # Check to see if we actioned into a terminal state
    if next_state_index >= number_of_non_terminal_states:
        state_action_value_function[updating_state_action_value_function_index, state_index, action_index] += alpha * (reward - state_action_value_function[updating_state_action_value_function_index, state_index, action_index])
    else:
        # Choose policy for chosen state by epsilon-greedy choosing from the state-action-value function */
        policy = epsilon_greedy_policy_from_state_action_function(max_number_of_actions, state_action_value_function, epsilon, next_state_index, policy)

        # Get next action
        next_action_index = np.random.choice(a = max_number_of_actions, p = policy[next_state_index, :])

        # Calculate state-action-function using quintuple SARSA
        state_action_value_function_indices = np.arange(number_of_state_action_value_functions)
        
        not_updating_state_action_value_function_index = np.random.choice(a = np.extract(condition = state_action_value_function_indices != updating_state_action_value_function_index, arr = state_action_value_function_indices))
        
        state_action_value_function[updating_state_action_value_function_index, state_index, action_index] += alpha * (reward + discounting_factor_gamma * state_action_value_function[not_updating_state_action_value_function_index, next_state_index, next_action_index] - state_action_value_function[updating_state_action_value_function_index, state_index, action_index])

        # Update state and action to next state and action
        state_index = next_state_index
        action_index = next_action_index
        
    return state_action_value_function, policy, state_index, action_index

In [60]:
def on_policy_temporal_difference_n_tuple_sarsa(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, number_of_state_action_value_functions, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length):
    for episode in range(0, number_of_episodes):
        # Initialize episode to get initial state and action
        initial_state_index, initial_action_index, policy = initialize_epsiode(number_of_non_terminal_states, max_number_of_actions, state_action_value_function, epsilon, policy)

        # Loop through episode and update the policy
        state_action_value_function, policy = loop_through_episode(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, number_of_state_action_value_functions, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length, initial_state_index, initial_action_index)
    
    return state_action_value_function, policy

# Run algorithm

In [61]:
# Print initial arrays
print("\nInitial state-action value function")
print(state_action_value_function)

print("\nInitial policy")
print(policy)

# Run on policy temporal difference n-tuple sarsa
state_action_value_function, policy = on_policy_temporal_difference_n_tuple_sarsa(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, number_of_state_action_value_functions, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length)

# Print final results
print("\nFinal state-action value function")
print(state_action_value_function)

print("\nFinal policy")
print(policy)


Initial state-action value function
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]

Initial policy
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 